### Import Packages

In [29]:
# work with geospatial data
import geopandas as gpd
import pandas as pd
from pyproj import Proj, Transformer

# work with files
import os
import yaml

# work with data
import xarray as xr

# work with time
import pytz

# util functions
from geospatial_utils import *

### Essential Variables

In [ ]:
# essential variables

# transform centroid to lat and lon
transformer = Transformer.from_crs(5070, 4326)
transformer_inv = Transformer.from_crs(4326, 5070)

# let's try to convert everything to EPSG:5070. For more accurate area measurements

# boundary for state of California
us_states = gpd.read_file("Data/Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp")
us_states.to_crs("EPSG:5070", inplace=True)
ca_state = us_states[us_states["STUSPS"] == "CA"]

# California counties
us_counties = gpd.read_file("Data/Boundaries/cb_2018_us_county_500k/cb_2018_us_county_500k.shp")
ca_counties = us_counties[us_counties["STATEFP"] == "06"]

# HUC8 subbasins
if not os.path.exists("Data/Boundaries/ca_HUC8.geojson"):
    huc8 = gpd.read_file("Data/Boundaries/HUC8_CONUS/HUC8_US.shp")
    huc8.to_crs("EPSG:5070", inplace=True)
    huc8['CA'] = huc8["STATES"].map(lambda x: "CA" in x)
    huc8 = huc8[huc8["CA"]]

    # intersect with California
    huc8_ca = gpd.clip(huc8, ca_state)

    # compute centroid locations
    huc8_ca['centroid'] = huc8_ca.to_crs("+proj=cea").centroid.to_crs(huc8_ca.crs)
    huc8_ca['centroid_lat_lon'] = huc8_ca['centroid'].map(lambda pt: transformer.transform(pt.x, pt.y))
    huc8_ca['centroid_lat'] = huc8_ca['centroid_lat_lon'].map(lambda x: x[0])
    huc8_ca['centroid_lon'] = huc8_ca['centroid_lat_lon'].map(lambda x: x[1])
    huc8_ca.drop(columns=['centroid', 'centroid_lat_lon'], inplace=True) # avoid multiple geometry columns to avoid confusion

    huc8_ca.to_file("Data/Boundaries/ca_HUC8.geojson", driver="GeoJSON")
else:
    huc8_ca = gpd.read_file("Data/Boundaries/ca_HUC8.geojson")

Convert electricity prices to HUC8 subbasin level

In [ ]:
ca_county_prices = gpd.read_file("Data/Boundaries/ca_counties.geojson")

ca_county_prices = ca_county_prices[~ca_county_prices["Electricity Price ($/MWh)"].isna()]

# convert electricity prices from county level to HUC8 level data
prices_county_to_huc8 = resolve_regions(ca_county_prices, huc8_ca, "Electricity Price ($/MWh)", "HUC8")

prices_county_to_huc8.convert_regions()

Compute solar capacity factor from DNI, DHI, GHI

Currently, we are keeping the time zone as UTC. We will be careful when doing the conversion to insolation on the solar panel though.

In [45]:
ca_solar = xr.open_dataset('Data/Solar/ca_irradiation.nc4')
# np.load("Data/Solar/ca_irradiation.npz")
# time_index = pd.DatetimeIndex(ca_solar['time_index'])

# delta_time = (time_index[1] - time_index[0]).total_seconds()/3600 # time step, in hours
delta_time = pd.Timedelta(ca_solar.Time.values[1] - ca_solar.Time.values[0]).total_seconds()/3600 # time step, in hours
agg_number = 24/delta_time # number of time steps to aggregate to get a full day

dni_subbasins = ca_solar['DNI']
dhi_subbasins = ca_solar['DHI']
ghi_subbasins = ca_solar['GHI']
time_offset = ca_solar.attrs['Timezone']
time_index = ca_solar.Time.values # recall that these time values are in UTC

In [ ]:
# calculate the beta angle at each point.

# CA_time = pytz.timezone('America/Los_Angeles')

beta_angles = np.zeros((time_index.shape[0], huc8_ca.shape[0]))

azimuth_angles = np.zeros((time_index.shape[0], huc8_ca.shape[0]))

# get local clock time
# curr_times = time_index.map(lambda x: x.astimezone(CA_time))

for idx, (_, row) in enumerate(huc8_ca.iterrows()):
    curr_betas = []
    curr_azimuths = []
    
    centroid_lat = row['centroid_lat']
    centroid_lon = row['centroid_lon']

    # longitude correction. local meridian is 120 degrees for California
    # TODO I wonder if we can generalize this - to easily calculate for any location
    longitude_add = 4*(120 - centroid_lon)

    # for stamp in curr_times:
    for stamp, offset in zip(time_index, time_offset):
        # get current LOCAL TIME hour and day
        stamp = stamp + pd.Timedelta(offset, unit='hour') # add the appropriate time offset 
        hour = stamp.timetuple().tm_hour + (1/60)*stamp.timetuple().tm_minute
        day = stamp.timetuple().tm_yday
               
        # adding factor: equation of time
        time_input = (360/364)*(day-81)
        E = 9.87*np.sin(2*time_input*(2*np.pi/360)) - 7.53*np.cos(time_input*(2*np.pi/360)) - 1.5*np.sin(time_input*(2*np.pi/360))
        
        # get the hour angle
        H = 15*(12 - (hour + longitude_add/60 + E/60))
        
        declination = 23.45*np.sin((360/365)*(day - 81)*(2*np.pi/360))

        sin_beta = np.cos(centroid_lat*(2*np.pi/360))*np.cos(declination*(2*np.pi/360))*np.cos(H*(2*np.pi/360)) + np.sin(centroid_lat*(2*np.pi/360))*np.sin(declination*(2*np.pi/360))

        beta = np.arcsin(sin_beta)*(360/(2*np.pi))

        curr_betas.append(beta)

        sin_azimuth = (np.cos(declination*(2*np.pi/360))*np.sin(H*(2*np.pi/360)))/np.cos(beta*(2*np.pi/360))

        azimuth = np.arcsin(sin_azimuth)*(360/(2*np.pi))

        curr_azimuths.append(azimuth)

    beta_angles[:, idx] = np.array(curr_betas) # in degrees

    azimuth_angles[:, idx] = np.array(curr_azimuths) # in degrees

In [ ]:
# putting it all together to calculate solar energy production

cos_theta = np.sqrt(1 - (np.cos(beta_angles*(2*np.pi/360)) * np.cos(azimuth_angles*(2*np.pi/360)))**2)

rho = 0.2 # a typical value for ground reflectivity

I_BC = dni_subbasins * cos_theta # beam insolation on collector

I_DC = dhi_subbasins * ((1 + (np.sin(beta_angles*(2*np.pi/360)) / cos_theta))/2) # diffuse insolation on collector

I_RC = ghi_subbasins * rho * ((1 - (np.sin(beta_angles*(2*np.pi/360)) / cos_theta))/2) # reflected insolation on collector

total_insolation = I_BC + I_DC + I_RC

In [ ]:
# aggregate the insolation values into days
total_insolation = total_insolation.reshape((ghi_subbasins.shape[0]//agg_number, agg_number, huc8_ca.shape[0])) # shape is (# days, # time steps in a day, # HUC8 subbasins)

# sum across axis 1 (the time steps in the day) to get daily insolation
total_insolation = total_insolation.sum(axis=1)*delta_time # W/(m^2 * day).

In [ ]:
# save solar data
insolation_df = pd.DataFrame(total_insolation)
insolation_df.to_csv("Data/Solar/insolation_CA.csv") # W/(m^2 * day)

In [ ]:
# now think about generation per watt of capacity, per day
insolation_df = pd.read_csv("Data/Solar/insolation_CA.csv", index_col=0)
insolation_df_per_watt = insolation_df/1000 # TODO verify this is correct. we divide by 1000 to get a per-watt measurement. https://www.greenlancer.com/post/solar-panel-wattage-output-explained#:~:text=A%20solar%20panel%20rating%20measures,sunlight%20at%201000W%2Fsquare%20meters.
insolation_df_per_watt.to_csv("Data/Solar/insolation_CA_per_watt.csv")

Convert Wind Speed to Wind Power

In [ ]:
ca_wind = np.load("Data/Wind/wind_speed_subbasin_CA.csv")

In [ ]:
# read data for GE1.5-77 wind turbine model. Source: https://nrel.github.io/turbine-models/DOE_GE_1.5MW_77.html

# power rating curve
GE_power_curve = pd.read_csv('Data/Wind/DOE_GE_1.5MW_77.csv')

rated_wind_speeds = GE_power_curve['Wind Speed [m/s]'].values
rated_wind_power = GE_power_curve['Power [kW]'].values

# other specs
with open("Data/Wind/DOE_GE_1.5MW_77.yaml", 'r') as f:
    turbine_specs = yaml.safe_load(f)
    cut_in = turbine_specs['cut_in_wind_speed']
    cut_out = turbine_specs['cut_out_wind_speed']
    rated_wind = turbine_specs['rated_wind_speed']
    rated_power = turbine_specs['rated_power']

In [ ]:
def wind_speed_to_energy(wind_speeds):
    """ 
    Computes wind power based on wind speed input. For wind turbine model GE1.5-77.

    Parameters
    ----------
        wind_speeds: np.ndarray
            Time series of wind speeds.

    Returns
    -------
        Time series of wind power capacity factors derived from wind speeds.
    """

    power_output = np.interp(wind_speeds, rated_wind_speeds, rated_wind_power) # interpolate between measured points for wind power

    # wind power output is zero, below the cut-in speed
    power_output = np.where(wind_speeds < cut_in, 0, power_output)

    # wind power output is zero, above the cut-out speed
    power_output = np.where(wind_speeds > cut_out, 0, power_output)

    # wind power output is maxed out, between rated wind speed and cut-out speed
    power_output = np.where((wind_speeds > rated_wind) & (wind_speeds < cut_out), 1, power_output)

    # convert power outputs into capacity factor (range: 0 to 1)
    capacity_factor = power_output/rated_power

    return capacity_factor